## Conservación de la energía

El objetivo es implementar el módulo de conservación de la energía copiando lo que hace Gabriela en C++

Consideraciones:  
* En el código de Gabriela el vector y tiene: y[1]-> radio, y[2]-> velocidad, y[3]-> temperatura (T_bc), y[4]-> nh2o, y[5]-> TLi
* Uso el modelo de Toegel (modelo 2) solamente para la parte de conductividad térmica

### Importo dependencias

In [ ]:
#Importo librerías



In [ ]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [ ]:
#Importo notebooks

### Funciones

### Derivada_temperatura


In [3]:
from parametros import alfaM, pvap0, n_species
from conductividad_termica import Conductividad1, Conductividad2
from funciones_qcas import n_tot
import numpy as np

import math
pi = math.pi

ModuleNotFoundError: No module named 'parametros'

In [ ]:
from parametros import var_termo

In [ ]:
def dkdt(y, n, dndt, dTdt, dT2dt, dntdt):
    #Calculo dk1dt y dk2dt y los guardo en las variables dk1dt y dk2dt (son números, no vectores)

    #Calculo k1 y k2
    k1 = np.zeros(n_species)
    k2 = np.zeros(n_species)
    Conductividad1(k1)
    Conductividad2(k2)

    #Calculo dk1dt

    #Calculo dk2dt

    dk1dt=0.0

    ntot = n_tot(n)

    T_bc = y[var_termo["T_bc"]]


    for i in range(n_species):
        dk1dt=dk1dt+k2[i]*n[i]/ntot*dTdt-(k1[i]+k2[i]*T_bc)*n[i]/ntot/ntot*dntdt+(k1[i]+k2[i]*T_bc)*dndt[i]/ntot

    dk2dt=0.0

    for i in range(n_species):
        dk2dt=dk2dt+k2[i]*n[i]/ntot*dT2dt;

    return dk1dt, dk2dt


In [ ]:
def calcula_dmdt(ntot):
    #Calcula dmdt

    dmdt=0.0

    icont=1;

    while(icont<=NumEsp){
        dmdt=dmdt+(Masa[icont]*dydt[Nvar2+icont])/ntot/Na;
        icont=icont+1;
    }

    dmdt=dmdt-m/ntot*dntdt;

    return dmdt


def calcula_dnpdt():
    #Calcula dnpdt
    dnpdt=dntdt/V-4.0*3.14159*ntot*y[1]*y[1]*y[2]/V/V;
    return dnpdt

def calcula_dlandadt():
    dlandadt= 4.0*3.14159*y[1]*y[1]*y[2]/(sqrt(2.0)*sigmap*ntot)-V/(sqrt(2.0)*sigmap*ntot*ntot)*dntdt;

    return dlandadt


In [ ]:
def calcula_dlthdt(R, v_R, Xi):
   #Calcula dlth1dt y dlth2dt
   if(v_R == 0):
      dlth1dt=v_R/pi
      dlth2dt=0.0
   else:
      if((R/pi)<(np.sqrt(R*Xi/abs(v_R)))):
         dlth1dt=v_R/pi
         dlth2dt=0.0
      else:
         dlth1dt=0.5*np.sqrt(Xi/R/v_R)*v_R
         dlth2dt=-0.5*np.sqrt(R*Xi/v_R/v_R/v_R)
      
   return dlth1dt, dlth2dt

In [ ]:
from parametros import Tinf
from conductividad_termica import longitud_termica, Conductividad_termica_k
from funciones_qcas import n_gas, n_tot
from parametros import ap, alfae, K, rapidez

In [ ]:
def calcula_mpp(y,n, dndt, dTdt, dT2dt, dntdt):
   #Calculo mpp1 y mpp2
   
   if(alfaM==0):
      mpp1=0.0
      mpp2=0.0
   else:
      #Asigno las variables termodinámicas
      v_R = y[var_termo["v_R"]]
      T_bc = y[var_termo["T_bc"]]

      #Calculo el nro de partículas no condensable
      ngas = n_gas(n)
      #Calculo el nro total de partículas
      ntot = n_tot(n)

      pvap=pvap0
      dpvapdt=0.0

      #Calculo dk1dt y dk2dt
      dk1dt, dk2dt = dkdt(y, n, dndt, dTdt, dT2dt, dntdt)

      #Calculo dmdt
      dmdt = calcula_dmdt()

      #Calculo dnpdt
      dnpdt = calcula_dnpdt()

      #Calculo dlandadt
      dlandadt = calcula_dlandadt()

      #Calculo dlth1dt y dlth2dt
      dlth1dt, dlth2dt = calcula_dlthdt()


      #Uso el modelo 2 (modelo de Toegel):

      #Calculo k
      k = Conductividad_termica_k(T_bc, n)

      #Calculo lth
      lth = longitud_termica(y,k,nh2o0, ngas)

      dqdTB=0.0
      dq1dt=dk1dt*(Tinf-T_bc)/lth-k/lth*dTdt-k/lth/lth*(Tinf-T_bc)*dlth1dt
      dq2dt=dk2dt*(Tinf-T_bc)/lth-k/lth*dT2dt-k/lth/lth*(Tinf-T_bc)*dlth2dt

      #Termina el uso del modelo 2



      dTB1dt=(dydt[5]-(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*(-1.0/np/np*dnpdt*np.sqrt(m/TB)*qpp+
         1.0/2.0/np*np.sqrt(1.0/m/TB)*dmdt*qpp+1.0/np*np.sqrt(m/TB)*dq1dt))/
         (1.0+(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*
         (-1.0/2.0/np*np.sqrt(m/TB/TB/TB)*qpp+1.0/np*np.sqrt(m/TB)*dqdTB));

      dTB2dt=-(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)/np*np.sqrt(m/TB)*dq2dt/
         (1.0+(2.0-ap*alfae)/2.0/K/alfae*np.sqrt(pi/2.0/K)*
         (-1.0/2.0/np*np.sqrt(m/TB/TB/TB)*qpp+1.0/np*np.sqrt(m/TB)*dqdTB));

      dpv1dt=(1.0-y[Nvar2+6]/ntot)*pg/ntot*dntdt+y[Nvar2+6]/ntot*dpgdt;

      dpv2dt=y[Nvar2+6]/ntot*dpg2dt;



      if(rapidez ==1):
         mpp1=alfaM/np.sqrt(2.0*pi*Rv)*(dpvapdt/np.sqrt(y[5])-0.5*pvap/pow(y[5],1.5)*dydt[5]-
         1.0/np.sqrt(TB)*dpv1dt+0.5*pv/pow(TB,1.5)*dTB1dt);

         mpp2=alfaM/np.sqrt(2.0*pi*Rv)*(-1.0/np.sqrt(TB)*dpv2dt+0.5*pv/pow(TB,1.5)*dTB2dt);
      else:
         Omega=mp/pv*np.sqrt(Rv*T_bc/2.0);

         Gama=exp(-Omega*Omega)-Omega*np.sqrt(3.14159)*(1.0-erf(Omega,N));

         mpp1=alfaM/np.sqrt(2.0*pi*Rv)*(dpvapdt/np.sqrt(y[5])-0.5*pvap/pow(y[5],1.5)*dydt[5]-
         mp/pv*(1.0-erf(Omega,N))*dpv1dt*np.sqrt(pi*Rv*T_bc/2.0/TB)+
         0.5*mp*(1.0-erf(Omega,N))*np.sqrt(pi*Rv/2.0/TB/T_bc)*dTdt-
         Gama/np.sqrt(TB)*dpv1dt+0.5*Gama*pv/pow(TB,1.5)*dTB1dt)/
         (1.0-alfaM/2.0*(1.0-erf(Omega,N))*np.sqrt(T_bc/TB));

         mpp2=alfaM/np.sqrt(2.0*pi*Rv)*(-mp/pv*(1.0-erf(Omega,N))*dpv2dt*np.sqrt(pi*Rv*T_bc/
         2.0/TB)+0.5*mp*(1.0-erf(Omega,N))*np.sqrt(pi*Rv/2.0/TB/T_bc)*dT2dt-
         Gama/np.sqrt(TB)*dpv2dt+0.5*Gama*pv/pow(TB,1.5)*dTB2dt)/
         (1.0-alfaM/2.0*(1.0-erf(Omega,N))*np.sqrt(T_bc/TB));


   return mpp1, mpp2









In [ ]:
from parametros import rol, cl, Pinf, sigma, mu, 
from parametros import var_termo
from funciones_qcas import rog

In [2]:
ps=0 #presion acustica



#Calculo mpp1 y mpp2
mpp1, mpp2 = calcula_mpp(y,n)



def cons_ener_dpB1dt(v_R, R):
        rog_ = rog(y)
        
        return dpgdt+2.0*sigma*v_R/R/R+4.0*mu*v_R/R/R*(v_R-mp/rol)+ 4.0*mu/R/rol*mpp1-2.0*mp*mpp1*(1.0/rol-1.0/rog_)-mp*mp/rog_/rog_*drogdt;

def cons_enerdpB2dt(R):
        rog_ = rog(y)
        return dpg2dt-4.0*mu/R+4.0*mu/R/rol*mpp2-2.0*mp*mpp2*(1.0/rol-1.0/rog_);





 dydt[4]=dpgdt; ??

#Asigno variables termodinámicas
mp = y[var_termo["mp"]]
v_R = y[var_termo["v_R"]]
R = y[var_termo["R"]]



dpB1dt = cons_ener_dpB1dt()
dpB2dt = cons_enerdpB2dt()

dydt[var_termo["v_R"]]=(1.0/rol*(1.0+v_R/cl)*(pB-ps-Pinf)+mp/rol*(v_R+mp/2.0/rol+mp*v_R/2.0/cl/rol)-
        3.0/2.0*v_R*v_R*(1.0-1.0/3.0*v_R/cl+2.0*mp/3.0/cl/rol)+R/cl/rol*dpB1dt+
        mpp1*R/rol*(1.0-v_R/cl+mp/cl/rol))/(R*(1.0-v_R/cl+mp/cl/rol)-
        mpp2*R/rol*(1.0-v_R/cl+mp/cl/rol)-R/cl/rol*dpB2dt);


NameError: name 'y' is not defined

In [ ]:
#dydt_v_R: dydt[var_termo["v_R"]]. Valor luego de aplicar la ecuación de evolución de R(t)

def derivada_energia(y, n, dydt, dydt_v_R):
    dydt[var_termo["T_bc"]] = dTdt+dT2dt*dydt_v_R


: 